In [1]:
import sys
import os
from pathlib import Path
current_dir = os.getcwd()  
project_root = os.path.dirname(current_dir)  
sys.path.insert(0, project_root) 
import numpy as np
import pandas as pd
from src.data.data_preprocessor import DataPreprocessor
from src.config.config import Config
data_preprocessor = DataPreprocessor()

def detect_and_remove_anomalies(df, target_col, country_col='Country Name', year_col='Year', 
                               z_threshold=2.0, ratio_threshold=2.0, 
                               min_years=3, return_anomalies=False):
    """
    检测并移除目标列中的异常值 - 改进版
    
    参数:
    - df: 数据框
    - target_col: 目标列名（工业废物量）
    - country_col: 国家列名
    - year_col: 年份列名
    - z_threshold: Z-score阈值，超过此值视为异常
    - ratio_threshold: 比值阈值，与相邻年份比值超过此值视为异常
    - min_years: 最少需要的年份数据量，少于此值的国家不进行异常检测
    - return_anomalies: 是否返回异常值信息
    
    返回:
    - 清洗后的数据框，如果return_anomalies=True，则同时返回异常值信息
    """
    # 复制数据框，避免修改原始数据
    df_clean = df.copy()
    
    # 存储所有异常记录
    all_anomalies = pd.DataFrame()
    
    # 按国家分组处理
    for country, country_data in df_clean.groupby(country_col):
        # 如果数据量太少，跳过异常检测
        if len(country_data) < min_years:
            continue
            
        # 按年份排序
        country_data = country_data.sort_values(year_col)
        
        # 方法1: 基于Z-score的异常检测
        z_scores = np.abs((country_data[target_col] - country_data[target_col].mean()) / country_data[target_col].std())
        z_anomalies = country_data[z_scores > z_threshold].index
        
        # 方法2: 基于相邻年份比值的异常检测
        country_data_sorted = country_data.sort_values(year_col)
        
        # 计算与前一年的比值
        prev_ratios = country_data_sorted[target_col] / country_data_sorted[target_col].shift(1)
        # 计算与后一年的比值
        next_ratios = country_data_sorted[target_col] / country_data_sorted[target_col].shift(-1)
        
        # 找出异常记录
        ratio_anomalies = country_data_sorted[
            ((prev_ratios > ratio_threshold) | (prev_ratios < 1/ratio_threshold)) & 
            ((next_ratios > ratio_threshold) | (next_ratios < 1/ratio_threshold))
        ].index
        
        # 方法3: 考虑年份间隔的异常检测
        # 计算年份间隔
        year_gaps = country_data_sorted[year_col].diff()
        # 调整比值阈值，根据年份间隔
        adjusted_ratios = prev_ratios / year_gaps
        gap_anomalies = country_data_sorted[
            (adjusted_ratios > ratio_threshold) & (year_gaps > 1)
        ].index
        
        # 方法4: 基于IQR的异常检测 (新增)
        Q1 = country_data[target_col].quantile(0.25)
        Q3 = country_data[target_col].quantile(0.75)
        IQR = Q3 - Q1
        iqr_lower_bound = Q1 - 1.5 * IQR
        iqr_upper_bound = Q3 + 1.5 * IQR
        iqr_anomalies = country_data[
            (country_data[target_col] < iqr_lower_bound) | 
            (country_data[target_col] > iqr_upper_bound)
        ].index
        
        # 方法5: 基于移动平均的异常检测 (新增)
        if len(country_data) >= 5:  # 至少需要5个数据点
            # 计算移动平均和标准差
            rolling_mean = country_data[target_col].rolling(window=3, min_periods=1).mean()
            rolling_std = country_data[target_col].rolling(window=3, min_periods=1).std().fillna(country_data[target_col].std())
            
            # 检测时间序列异常值
            lower_bound = rolling_mean - 2 * rolling_std
            upper_bound = rolling_mean + 2 * rolling_std
            
            # 创建异常值掩码
            rolling_anomalies = country_data[
                (country_data[target_col] < lower_bound) | 
                (country_data[target_col] > upper_bound)
            ].index
        else:
            rolling_anomalies = pd.Index([])
        
        # 方法6: 基于增长率的异常检测 (新增)
        # 计算年增长率
        growth_rates = country_data[target_col].pct_change() * 100
        # 异常增长率阈值 (例如超过50%或下降超过30%)
        growth_anomalies = country_data[
            (growth_rates > 50) | (growth_rates < -30)
        ].index
        
        # 综合多种方法，确定最终的异常值
        # 使用投票机制：至少被两种方法判定为异常的记录
        anomaly_counts = {}
        for idx in set(z_anomalies) | set(ratio_anomalies) | set(gap_anomalies) | set(iqr_anomalies) | set(rolling_anomalies) | set(growth_anomalies):
            anomaly_counts[idx] = 0
            if idx in z_anomalies: anomaly_counts[idx] += 1
            if idx in ratio_anomalies: anomaly_counts[idx] += 1
            if idx in gap_anomalies: anomaly_counts[idx] += 1
            if idx in iqr_anomalies: anomaly_counts[idx] += 1
            if idx in rolling_anomalies: anomaly_counts[idx] += 1
            if idx in growth_anomalies: anomaly_counts[idx] += 1
        
        # 至少被两种方法判定为异常
        anomaly_indices = [idx for idx, count in anomaly_counts.items() if count >= 2]
        
        if anomaly_indices and return_anomalies:
            # 收集异常记录信息
            anomalies = country_data.loc[anomaly_indices].copy()
            anomalies['z_score'] = z_scores.loc[anomaly_indices]
            anomalies['prev_ratio'] = prev_ratios.loc[anomaly_indices]
            anomalies['next_ratio'] = next_ratios.loc[anomaly_indices]
            anomalies['year_gap'] = year_gaps.loc[anomaly_indices]
            # 添加新的异常检测指标
            anomalies['iqr_lower'] = iqr_lower_bound
            anomalies['iqr_upper'] = iqr_upper_bound
            if len(country_data) >= 5:
                anomalies['rolling_lower'] = lower_bound.loc[anomaly_indices]
                anomalies['rolling_upper'] = upper_bound.loc[anomaly_indices]
            anomalies['growth_rate'] = growth_rates.loc[anomaly_indices]
            anomalies['anomaly_methods'] = [anomaly_counts[idx] for idx in anomaly_indices]
            all_anomalies = pd.concat([all_anomalies, anomalies])
        
        # 从清洗数据中移除异常值
        df_clean = df_clean.drop(anomaly_indices)
    
    if return_anomalies:
        return df_clean, all_anomalies
    else:
        return df_clean

In [ ]:
"""加载1990-2022数据"""
historical_df = pd.read_excel(
    Config.FEATURE_CONFIG['historical_data_path'],
    sheet_name=Config.FEATURE_CONFIG['historical_sheet'],
    usecols=Config.FEATURE_CONFIG['usecols'] 
)

historical_df

In [ ]:
"""执行完整处理流程"""
# 第一阶段：处理历史数据
all_countries_df = data_preprocessor.process_historical_data(historical_df)
features_path = Path(Config.PATH_CONFIG['features_dir']) / 'global_features.csv'
all_countries_df.to_csv(features_path, index=False)

In [ ]:
# 第二阶段：处理MSW数据
"""加载包含MSW的目标数据"""
msw_df = pd.read_excel(
    Config.FEATURE_CONFIG['historical_msw_data_path'],
    sheet_name=Config.FEATURE_CONFIG['historical_msw_sheet'],
    usecols=Config.FEATURE_CONFIG['usecols'] + [Config.DATA_CONFIG['target_column']]
)

# 先检测异常值但不移除，查看异常值情况
target_col = Config.DATA_CONFIG['target_column']
_, anomalies = detect_and_remove_anomalies(msw_df, target_col, return_anomalies=True)
print(f"检测到 {len(anomalies)} 条异常记录:")
print(f"检测到 {len(anomalies)} 条异常记录:")
if len(anomalies) > 0:
    display(anomalies.sort_values(['Country Name', 'Year']))
else:
    print("没有检测到异常记录")

# 分析每个国家数据的分布情况
print("\n===== 各国数据分布情况分析 =====")
for country, country_data in msw_df.groupby('Country Name'):
    if len(country_data) < 3:  # 跳过数据量太少的国家
        continue
        
    country_data = country_data.sort_values('Year')
    
    # 计算Z-score
    z_scores = np.abs((country_data[target_col] - country_data[target_col].mean()) / country_data[target_col].std())
    
    # 计算年度增长比例
    growth_ratios = country_data[target_col].pct_change()
    
    print(f"\n国家: {country}, 数据点数: {len(country_data)}")
    print(f"平均值: {country_data[target_col].mean():.2f}, 标准差: {country_data[target_col].std():.2f}")
    print(f"最大Z-score: {z_scores.max():.2f}")
    print(f"最大年度增长率: {growth_ratios.max():.2%}")
    print(f"最小年度增长率: {growth_ratios.min():.2%}")
    
    # 显示可能的异常点
    potential_anomalies = country_data[(z_scores > 2.0) | (growth_ratios > 0.5) | (growth_ratios < -0.3)]
    if len(potential_anomalies) > 0:
        print("潜在异常点:")
        display(potential_anomalies[['Year', target_col]])


In [ ]:
df_clean = detect_and_remove_anomalies(msw_df, target_col)
print(f"原始数据: {len(msw_df)} 行, 清洗后: {len(df_clean)} 行")

In [ ]:
train_df, predict_df = data_preprocessor.merge_features(df_clean)
print(f"用于训练的数据: {len(train_df)} 行, 预测的历史数据: {len(predict_df)} 行, 总共: {len(train_df)+len(predict_df)}行")

In [ ]:
# 保存最终数据集
train_df.to_csv(
    Path(Config.PATH_CONFIG['features_dir']) / 'training_data.csv', 
    index=False,
    encoding='utf-8-sig'
)
predict_df.to_csv(
    Path(Config.PATH_CONFIG['features_dir']) / 'prediction_data.csv', 
    index=False,
    encoding='utf-8-sig'
)

In [2]:
"""加载2022-2050数据"""
future_df = pd.read_excel(
    Config.FEATURE_CONFIG['future_data_path'],
    sheet_name=Config.FEATURE_CONFIG['future_sheet'],
    usecols=Config.FEATURE_CONFIG['usecols']+['Scenario']
)

"""加载1990-2022数据"""
historical_df = pd.read_excel(
    Config.FEATURE_CONFIG['historical_data_path'],
    sheet_name=Config.FEATURE_CONFIG['historical_sheet'],
    usecols=Config.FEATURE_CONFIG['usecols']
)

# !! 新增：在调用处理函数前，预先移除未来数据中与历史数据重叠的年份 !!
last_historical_year = historical_df['Year'].max()
print(f"历史数据最后一年: {last_historical_year}")
print(f"过滤前未来数据形状: {future_df.shape}")

future_df_filtered = future_df[future_df['Year'] > last_historical_year].copy()
print(f"过滤掉年份 <= {last_historical_year} 后，未来数据形状: {future_df_filtered.shape}")

# 检查过滤后是否还有数据
if future_df_filtered.empty:
    print("错误：过滤重叠年份后，没有有效的未来数据。请检查数据范围。")
    # 可以选择退出或抛出异常
    # sys.exit(1) 
else:
    # 使用过滤后的 future_df 调用处理函数
    processed_data_paths = data_preprocessor.process_future_data(historical_df, future_df_filtered)
    print("未来数据特征生成完成。各场景文件路径:")
    print(processed_data_paths)

历史数据最后一年: 2022
过滤前未来数据形状: (66755, 9)
过滤掉年份 <= 2022 后，未来数据形状: (65910, 9)
加载特征工程参数从: e:\code\jupyter\固废产生\SW-Prediction\iw\src\features\featurefile\feature_params.pkl
特征工程参数已从 e:\code\jupyter\固废产生\SW-Prediction\iw\src\features\featurefile\feature_params.pkl 加载
合并历史数据和未来数据...
合并后数据形状: (71341, 9)
按国家和年份排序...
应用特征工程转换...
开始 Transform 过程...
应用对数变换...
...已生成: GDP PPP 2017_log
...已生成: GDP PPP/capita 2017_log
...已生成: Population_log
创建非线性特征...
...已生成: gdp_pc_log_squared
创建时间特征...
...已生成: year_relative (基准年: 1990)
计算增长率...
...已生成: GDP PPP 2017_log_growth_1y
...已生成: Population_log_growth_1y
处理城市化率...
...已生成: urban_pop_perc
创建交互特征...
...已生成: gdp_log_x_pop_log
...已生成: gdp_pc_log_x_urban
...已生成: gdp_pc_log2_x_urban
创建时间相关的交互特征...
...已生成: year_relative_x_gdp_pc_log
...已生成: year_relative_x_Population_log
...已生成: year_relative_x_urban_pop_perc
处理缺失值和无穷值...
...正在使用中位数填充NaN值...
...填充完成，共处理了 0 个 NaN 值。
裁剪极端数值...
Transform 过程完成。
特征转换后数据形状: (71341, 23)
筛选未来数据...
筛选出的未来数据形状: (65910, 23)
按场景拆分并保存未来特征文件...
  处理